# TFX on AI Platform - Pipeline

In [25]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
import tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.3.2
TFX version: 0.28.0


In [26]:
from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [27]:
PIPELINE_NAME = 'my-pipeline'

# Make the Directory Structure for the Pipeline
LOCAL_DIR = PIPELINE_NAME
DATA_PATH = 'gs://statmike-mlops/digits/data'
TRAINER_PATH = '{}/trainer'.format(LOCAL_DIR)
PIPELINE_PATH = '{}/pipeline'.format(LOCAL_DIR)
METADATA_PATH = '{}/metadata/metadata.db'.format(LOCAL_DIR)
MODEL_PATH = '{}/model'.format(LOCAL_DIR)

In [28]:
!rm -rf {LOCAL_DIR}
!mkdir {LOCAL_DIR}
!mkdir {TRAINER_PATH}

In [29]:
%%writefile {TRAINER_PATH}/task.py
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys

parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--model-dir', dest='model_dir', default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--epochs',dest='epochs', default=10, type=int, help='Number of Epochs')
parser.add_argument('--batch_size',dest='batch_size', default=32, type=int, help='Batch Size')
#parser.add_argument('',dest='', default=, type=, help='')
args = parser.parse_args()

# built in parameters for data source:
#PROJECT_ID='statmike-mlops'
#BQDATASET_ID='digits'
#BQTABLE_ID='digits_prepped'

selected_fields = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target']
output_types = ['FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'INT64']

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

#client = BigQueryClient()
#session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
#table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(args.batch_size)

#client = BigQueryClient()
#session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
#table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(args.batch_size)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
#tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

history = model.fit(train,epochs=args.epochs)

model.save(args.model_dir)

Writing my-pipeline/trainer/task.py


In [ ]:
from tfx.components import CsvExampleGen
from tfx.extensions.google_cloud_big_query.example_gen import component as big_query_example_gen_component
from tfx.components import Pusher
from tfx.components import Trainer
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.orchestration.local import local_dag_runner


from typing import Optional, List, Text
import os

In [ ]:
GOOGLE_CLOUD_REGION = 'us-central1'
GOOGLE_CLOUD_PROJECT='statmike-mlops'
GCS_BUCKET_NAME=GOOGLE_CLOUD_PROJECT
BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS = [
    '--project=' + GOOGLE_CLOUD_PROJECT,
    '--temp_location=' + os.path.join('gs://', GCS_BUCKET_NAME, 'tmp'),
    ]
BIG_QUERY_QUERY = """
      SELECT * FROM `statmike-mlops.digits.digits_prepped`
  """

In [34]:
def mypipeline(pipeline_name: str, pipeline_path: str, data_path: str, query: str,
                     train_file: str, model_path: str,
                     metadata_path: str, beam_pipeline_args: Optional[List[Text]] = None) -> pipeline.Pipeline:

  # Brings data into the pipeline.
  #example_gen = CsvExampleGen(input_base=data_path)
  example_gen = big_query_example_gen_component.BigQueryExampleGen(query=query)
    
  # Uses user-provided Python function that trains a model.
  trainer = Trainer(
      module_file=train_file,
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
      examples=example_gen.outputs['examples'],
      train_args=trainer_pb2.TrainArgs(num_steps=100),
      eval_args=trainer_pb2.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = Pusher(
      model=trainer.outputs['model'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=model_path)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      #trainer,
      #pusher,
  ]

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_path,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(metadata_path),
      components=components,
      beam_pipeline_args=beam_pipeline_args)

In [35]:
local_dag_runner.LocalDagRunner().run(mypipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_path=PIPELINE_PATH,
    data_path=DATA_PATH,
    query = BIG_QUERY_QUERY,
    train_file=TRAINER_PATH+'/task.py',
    model_path=MODEL_PATH,
    beam_pipeline_args=BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS,
    metadata_path=METADATA_PATH
))

INFO:absl:Running pipeline:
 pipeline_info {
  id: "my-pipeline"
}
nodes {
  pipeline_node {
    node_info {
      type {
        name: "tfx.extensions.google_cloud_big_query.example_gen.component.BigQueryExampleGen"
      }
      id: "BigQueryExampleGen"
    }
    contexts {
      contexts {
        type {
          name: "pipeline"
        }
        name {
          field_value {
            string_value: "my-pipeline"
          }
        }
      }
      contexts {
        type {
          name: "pipeline_run"
        }
        name {
          field_value {
            string_value: "2021-04-15T12:15:12.374768"
          }
        }
      }
      contexts {
        type {
          name: "node"
        }
        name {
          field_value {
            string_value: "my-pipeline.BigQueryExampleGen"
          }
        }
      }
    }
    outputs {
      outputs {
        key: "examples"
        value {
          artifact_spec {
            type {
              name: "Examples"
   